In [0]:
dbutils.library.installPyPI("geopandas")
dbutils.library.installPyPI("shapely")

PyPI package geopandas has been installed already. The previously installed package is `geopandas`. To resolve this issue, detach and re-attach the notebook to create a new environment or rename the package.
PyPI package shapely has been installed already. The previously installed package is `shapely`. To resolve this issue, detach and re-attach the notebook to create a new environment or rename the package.
Out[176]: False

In [0]:
import pyspark
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, DateType, IntegerType, FloatType
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.utils import AnalysisException
import pyspark.sql.functions as F
import pandas as pd
from shapely.geometry import Point, Polygon, shape
from shapely import wkb, wkt
import geopandas as gpd
import numpy as np

In [0]:
#TAREA 2, PARTE 1 PUNTO A
#esctructura para datos que estan desde 2017 para delante
df_schema = StructType() \
            .add(field= "VendorID", data_type=StringType(), nullable= True) \
            .add(field="tpep_pickup_datetime", data_type=DateType(), nullable= True) \
            .add(field="tpep_dropoff_datetime", data_type=DateType(), nullable= True) \
            .add(field="passenger_count", data_type=IntegerType(), nullable= True) \
            .add(field="trip_distance", data_type=DoubleType(), nullable= True) \
            .add(field="RatecodeID", data_type=IntegerType(), nullable= True) \
            .add(field="store_and_fwd_flag", data_type=StringType(), nullable= True) \
            .add(field="PULocationID", data_type=IntegerType(), nullable= True) \
            .add(field="DOLocationID", data_type=IntegerType(), nullable= True) \
            .add(field="payment_type", data_type=IntegerType(), nullable= True) \
            .add(field="fare_amount", data_type=DoubleType(), nullable= True) \
            .add(field="extra", data_type=DoubleType(), nullable= True) \
            .add(field="mta_tax", data_type=DoubleType(), nullable= True) \
            .add(field="tip_amount", data_type=DoubleType(), nullable= True) \
            .add(field="tolls_amount", data_type=DoubleType(), nullable= True) \
            .add(field="improvement_surcharge", data_type=DoubleType(), nullable= True) \
            .add(field="total_amount", data_type=DoubleType(), nullable= True) \
            .add(field="congestion_surcharge", data_type=DoubleType(), nullable= True)

#eschema para datos de 2013 hasta 2017
df_schema_1 = StructType() \
            .add(field= "VendorID", data_type=StringType(), nullable= True) \
            .add(field="tpep_pickup_datetime", data_type=DateType(), nullable= True) \
            .add(field="tpep_dropoff_datetime", data_type=DateType(), nullable= True) \
            .add(field="passenger_count", data_type=IntegerType(), nullable= True) \
            .add(field="trip_distance", data_type=DoubleType(), nullable= True) \
            .add(field="pickup_longitude", data_type=FloatType(), nullable= True) \
            .add(field="pickup_latitude", data_type=FloatType(), nullable= True) \
            .add(field="RatecodeID", data_type=IntegerType(), nullable= True) \
            .add(field="store_and_fwd_flag", data_type=StringType(), nullable= True) \
            .add(field="dropoff_longitude", data_type=FloatType(), nullable= True) \
            .add(field="dropoff_latitude", data_type=FloatType(), nullable= True) \
            .add(field="payment_type", data_type=IntegerType(), nullable= True) \
            .add(field="fare_amount", data_type=DoubleType(), nullable= True) \
            .add(field="extra", data_type=DoubleType(), nullable= True) \
            .add(field="mta_tax", data_type=DoubleType(), nullable= True) \
            .add(field="tip_amount", data_type=DoubleType(), nullable= True) \
            .add(field="tolls_amount", data_type=DoubleType(), nullable= True) \
            .add(field="improvement_surcharge", data_type=DoubleType(), nullable= True) \
            .add(field="total_amount", data_type=DoubleType(), nullable= True)

def has_column(df, col):
    try:
        df[col]
        return True
    except AnalysisException:
        return False
            
file_location = "dbfs:/FileStore/tables/yellow_tripdata_2016_03.csv"

df_prueba = spark.read.format("csv") \
      .option("header", True) \
      .load(file_location)

if (has_column(df_prueba, "PULocationID")):
  print("primero")
  df_with_schema = spark.read.format("csv") \
      .option("header", True) \
      .schema(df_schema) \
      .load(file_location)
  df_with_schema_col = df_with_schema.withColumn("year", year(df_with_schema.tpep_pickup_datetime)) \
              .withColumn("month", month(df_with_schema.tpep_pickup_datetime)) \
              .withColumn("day", dayofmonth(df_with_schema.tpep_pickup_datetime)) 
			  
  #display(df_with_schema_col)
else:
  print("segundo")
  df_with_schema = spark.read.format("csv") \
      .option("header", True) \
      .schema(df_schema_1) \
      .load(file_location)
  df_with_schema_col = df_with_schema.withColumn("year", year(df_with_schema.tpep_pickup_datetime)) \
              .withColumn("month", month(df_with_schema.tpep_pickup_datetime)) \
              .withColumn("day", dayofmonth(df_with_schema.tpep_pickup_datetime)) 
			  
  #display(df_with_schema_col)
  
#display(df_with_schema_col.groupBy('PULocationID').count())

#pu_location_id = df_with_schema_col.select("PULocationID").collect()






segundo

In [0]:
#PARTE 1 PUNTO B
file_location = "/FileStore/tables/nyu_2451_36743_geojson.json"
data = spark.read.json(file_location)
location_id = data.select("features.properties.locationid")
segment_location_id = location_id.select("locationid").collect()
lista_id = segment_location_id[0][0] #Lista de ID_Zonas extraidas del json

name_location = data.select("features.properties.zone")
segment_name_location = name_location.select("zone").collect()
lista_name = segment_name_location[0][0] #Lista de nombres extraidas del json 

geometry_zone = data.select("features.geometry.coordinates")
segment_geometry_zone = geometry_zone.select("coordinates").collect()


def transform_coma(algo):
  lista_final = []
  for h in algo:
    st_h = str(h)
    new_ok = st_h.replace(",", "").replace("[","").replace("]","")
    lista_final.append(new_ok)
  return lista_final

#Arregral el jason para usar geopandas
#quitar las comas de dentro
geo_without_coma =[transform_coma(segment_geometry_zone[0][0][i][0][0]) for i in range(len(segment_geometry_zone[0][0]))]

#reemplazar brackets por parentesis
lista_geometry_zone = ["Multipolygon " + str([[geo_without_coma[i]]]).replace("[","(").replace("]",")").replace("'","") for i in range(len(lista_geometry_zone)) ]

id_name_coordinate_tuple = [(lista_id[i],lista_name[i],lista_geometry_zone[i]) for i in range(len(lista_name))] 
data_id_zone_coordinates = {
  'id': lista_id,
  'zone_name': lista_name,
  'geom': lista_geometry_zone
}
df_id_zone_coordinates = pd.DataFrame(data_id_zone_coordinates, columns=['id', 'zone_name', 'geom']) #Dataframe para utilizar con geopandas

def name_zone(value_1, lista = id_name_coordinate_tuple):
  for i,j,k in lista:
    if(i==value_1):
      return j
  return 'Locacion NO registrada'

  
  
def id_zone(value_1, lista = id_name_coordinate_tuple):
  for i,j,k in lista:
    if(j==value_1):
      return i
  return 'Locacion NO registrada'
  
id_zone_UDF = F.udf(lambda x:id_zone(x), IntegerType())
name_zone_UDF = F.udf(lambda x:name_zone(x))
  
  
if (has_column(df_with_schema_col, "PULocationID")):
  df_final = df_with_schema_col.withColumn('PULocationZone',name_zone_UDF(df_with_schema_col.PULocationID))\
                  .withColumn('DOLocationZone',name_zone_UDF(df_with_schema_col.DOLocationID))
  #display(df_final)

else:
  
  df_id_zone_coordinates['geom']=df_id_zone_coordinates['geom'].apply(wkt.loads)
  gdf  = gpd.GeoDataFrame(df_id_zone_coordinates, geometry='geom')
    
  def find_borough(latitude, longitude): 
    mgdf = gdf.apply(lambda x: x['zone_name'] if x['geom'].intersects(Point(longitude,latitude)) else None, axis=1)
    idx = mgdf.first_valid_index()
    first_valid_value = mgdf.loc[idx] if idx is not None else None
    return first_valid_value
    
  find_borough_udf = F.udf(lambda x,y: find_borough(x,y))
  df_raw_PU = df_with_schema_col.withColumn("pickup_zone", find_borough_udf(df_with_schema_col.pickup_latitude,df_with_schema_col.pickup_longitude))\
    .withColumn("dropoff_zone", find_borough_udf(df_with_schema_col.dropoff_latitude,df_with_schema_col.dropoff_longitude))
    
    
  df_final = df_raw_PU.withColumn("PULocationID", id_zone_UDF(df_raw_PU.pickup_zone))\
            .withColumn("DOLocationID", id_zone_UDF(df_raw_PU.dropoff_zone))
  #display(df_final)


In [0]:

pu_location_list = [row.PULocationID for row in df_final.select('PULocationID').collect()]
#pu_tuple = [(pu_location_list_id[i],pu_location_list_name) for i in range(len(pu_location_list_name))]
do_location_list = [row.DOLocationID for row in df_with_schema_col.select('DOLocationID').collect()]
matriz_OD_zeros = np.zeros([len(lista_id),len(lista_name)])

for i in range(1, len(pu_location_list)):
  if i>263:
    pass
  else:
    matriz_OD_zeros[pu_location_list[i]][do_location_list[i]] +=1 
    
data = pd.DataFrame(matriz_OD_zeros)

In [0]:
#Punto 2D
frecuent_places = df_final.groupBy('DOLocationID').count().orderBy('count',ascending=False)
frecuent_places_names = frecuent_places.withColumn('NameLocation',name_zone_UDF(frecuent_places.DOLocationID))

